<a href="https://colab.research.google.com/github/KhanShaheb34/RabindraAI/blob/main/RabindraAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RabindraAI

A text generator trained on poems of Rabindranath Tagore

Dataset taken from [Kaggle - Rabindranath Tagore Online Variorum](https://www.kaggle.com/nrkapri/rabindranath-tagore-online-variorum)

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random

## Reading the dataset

In [1]:
text = open('poems.txt').read()[:500000]
chars = list(set(text))
data_size, vocab_size = len(text), len(chars)
char_indices = { ch:i for i,ch in enumerate(chars) }
indices_char = { i:ch for i,ch in enumerate(chars) }

## Preparing the dataset

Here we will be giving 40 characters as input to our model and try to guess the next character.

So we are createing two variable `x` and `y` to save the input and output.

In [3]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)))
y = np.zeros((len(sentences), len(chars)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 166654


## Model

We will use a two layer LSTM for this task

In [4]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(maxlen, len(chars))),
        layers.LSTM(128, return_sequences=True),
        layers.LSTM(128),
        layers.Dropout(0.2),
        layers.Dense(len(chars), activation="softmax")
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 128)           120832    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 107)               13803     
Total params: 266,219
Trainable params: 266,219
Non-trainable params: 0
_________________________________________________________________


## Sample Method

In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Training 

In [ ]:
epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("==> Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0]:
        print("==> Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('==> Generating with seed:\n' + sentence)
        in_sentence = sentence

        for i in range(300):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("==> Generated: \n", in_sentence + generated)
        print()

## Testing the Model

We can give our model a sentence of atmost 40 characters as input. And it will generate a poem based on the input.

In [20]:
generated = ""
diversity = 0.5
sentence = "দেখেছি তোমার ওই দু চো"
if len(sentence) > maxlen:
  sentence = sentence[:maxlen]
else:
  pad = " " * (maxlen - len(sentence))
  sentence = pad + sentence
print('==> Generating with seed:\n' + sentence)
in_sentence = sentence

for i in range(400):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    sentence = sentence[1:] + next_char
    generated += next_char

print("==> Generated: \n", in_sentence + generated)
print()

==> Generating with seed:
                   দেখেছি তোমার ওই দু চো


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


==>Generated: 
                    দেখেছি তোমার ওই দু চোখে,
মানা মনে বাঁধি মনে যে সে কানে বেঁশি।
কোনো বাঁধি সে কে দেখে দিল মনে,
বাঁধা বিদ্রোহ বালান
নিত্য আঁখি হতে আকাশ সে বিচ্ছুক্ত প্রদ্য
বিজন বালুকায়ে
সহসা মিলি দিবে সাথ।
বিরহ বিস্তানিতেছিলে তারে আমার ভাঙিয়া।
সেই মনো বুঝি বাঁধনে যে বাণী
বসন্ত কুজ নশীর্য বুলায়।
দিনে নিদ্রায় আসি
কাঁপা দিয়ে উঠে বাঁধয়ি
কিন্দে তার স্বর্গ তার
সেই তো না কোহার
তারে যায় বাজে মনে
বিদায়ের বাঁধন তবে
আকাশে তার করি তারা
মেলে দিয়ে আম



## Further Reading

- [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)